In [1]:
import numpy as np

from UQpy.RunModel import RunModel
from UQpy.Distributions import Normal
from UQpy.Surrogates import Kriging
from UQpy.SampleMethods import AKMCS, LHS, MCS

Next we create a `RunModel` instance, which will define how UQpy will interface to our simulation.  Here we simply pass in the names for the script that runs the model, the input file template, the variable names (which are referenced by the input file template), the output script, and the function within the output script that should be called to extract results:

In [2]:
model = RunModel(model_script='nats_uqpy_script.py',
                 input_template='nats_input.txt',
                 var_names=['cont_resp'],
                 output_script='nats_output_script.py',
                 output_object_name='get_output')

Next we define the input random variables.  For our example, the random variables are the latitude and longitude coordinates associated with a given waypoint for the first aircraft.  For demonstration purposes, we will assign them both normal distributions with a standard deviation of 1:

In [3]:
rvs = [Normal(15, 3)]

We must create an instance of the surrogate model class for use by UQpy in the `AKMCS` sampling method.  UQpy supports use of either its own Kriging surrogate model or the one provided by scikit-learn.  We will use the built-in UQpy Kriging class.  We define a constant trend function, the usual squared-exponential ("Gaussian") correlation function, and starting values for the correlation parameters:

In [4]:
surrogate = Kriging(reg_model='Constant', corr_model='Gaussian', corr_model_params = [1.0])

We are now ready to run the AKMCS method in UQpy.  We will define the limit state value as a separation distance of `Z0 = 0.1` (this is for demonstration purposes only and is not physically meaningful).  The `AKMCS` class in UQpy is designed to accept the random variable deinition, the `RunModel` instance, the Kriging surrogate model instance, along with values for sample sizes and other options.  In principle, we should be able to use the `nstart` option to specify the number of initial samples to use, but there is a bug in the current version of the code (a bug report has been filed), so it is necssary to pass in the values of the initial samples.  We do this by generating the initial samples using the `LHS` method with a sample size of 6 (for repeatability, we also set the random seed using NumPy).

`AKMCS` is actually a general adapative sampling method that supports different "learning functions", one of which is the expected feasability function (EFF) used by the EGRA method.  We specify this via the `learning_function='EFF'` option.  This learning function has other required options that must be passed to the `AKMCS` init function:
* `eff_a`: The limit state value
* `eff_epsilon`: The $\varepsilon$ parameter in the EGRA algorithm.  Unfortunately, it appears that UQpy defines this in an aboslute sense, whereas the recommendation from the original EGRA paper is to use $\varepsilon = 2 \sigma_G(x)$, where $\sigma_G(x)$ is the GP prediction standard deviation.  For demonstration, we use a value of $\varepsilon=0.1$, but an issue or pull request should probably be submitted to UQpy for support of defining $\varepsilon$ in terms of $\sigma_G$.
* `eff_stop`: Stopping criterion for the EFF.  We use the same value reported in the original EGRA paper.

As soon as we call the `AKMCS` constructor, that will trigger the AKMCS algorithm to run.  It will collect simulation data, updating the Kriging surrogate model, until the EFF stopping criterion is met.  Note that UQpy will also create a directory structure (named as `Model_Runs*`) that will store copies of all input and output files for each run.

In [5]:
# Limit state value
Z0 = 0.1

# Generate the initial samples ourselves, due to UQpy bug with nstart (samples=None) input
np.random.seed(1)
lhs = LHS(rvs, 6)

method = AKMCS(rvs, model, surrogate, nsamples=15, samples=lhs.samples,
               learning_function='EFF', eff_a=Z0, eff_epsilon=0.1, eff_stop=0.001)

ValueError: array must not contain infs or NaNs

Once the AKMCS method converges, the Kriging model instance `surrogate` contains the updated surrogate model.  We can inspect the sample input and output values (recall that 6 randomly chosen initial samples were used, and the remaining samples were selected based on maximization of the EFF):

In [6]:
print('Input samples:')
print(surrogate.samples)
print('Output samples:')
print(surrogate.values)

Input samples:
[[15.37993084]
 [10.56150678]
 [13.31102914]
 [13.70797538]
 [18.09291695]
 [16.49713349]]
Output samples:
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


Next, we create a plot of the 2D response surface by leveraging the `ResponseModel` base class in PARA-ATM, which provides general plotting functionality.

In [7]:
import matplotlib.pyplot as plt
from paraatm.rsm.base import ResponseSurface

# Create a class that derives from the `ResponseSurface` base class:
class UqpyKriging(ResponseSurface):
    def __init__(self, kriging):
        self.kriging = kriging
        super().__init__(kriging.samples, kriging.values)

    def __call__(self, X, return_stdev=False):
        if return_stdev:
            raise ValueError('not yet implemented')
        return self.kriging.predict(X)

# Create an instance and use the `surface_plot` method from the base class:
base_surrogate = UqpyKriging(surrogate)
base_surrogate.surface_plot()
plt.xlabel('Latitude')
plt.ylabel('Longitude')

RuntimeError: surface plot not valid for 1D response surfaces

Finally, we can use the converged Kriging surrogate model to efficiently estimate the probability of the separation distance falling below the given threshold using Monte Carlo sampling:

In [10]:
mcs = MCS(rvs, 100000)
preds = surrogate.predict(mcs.samples)
#print('POF:', sum(preds<Z0) / len(preds))

ValueError: einstein sum subscripts string contains too many subscripts for operand 1

In [9]:
import matplotlib.pyplot as plt

plt.hist(preds,bins=20)

NameError: name 'preds' is not defined